Source of this notebook: http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.YarxOfHML7F

### Gensim Word2Vec Tutorial: An End-to-End Example
By Kavita Ganesan / AI Implementation, Gensim, Hands-On NLP, Neural Embeddings

Modifications by Ali Arsanjani, could not clone github , errors 

This Notebook has been chosen and modified to provide examples for the <b> name your topic </b> assignment

### Imports and logging

First, we start with our imports and get logging established:

In [1]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [2]:
data_file="reviews_data.txt.gz"

with gzip.open ('reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [3]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2021-12-03 21:05:08,743 : INFO : reading file reviews_data.txt.gz...this may take a while
2021-12-03 21:05:08,745 : INFO : read 0 reviews
2021-12-03 21:05:10,422 : INFO : read 10000 reviews
2021-12-03 21:05:12,180 : INFO : read 20000 reviews
2021-12-03 21:05:14,253 : INFO : read 30000 reviews
2021-12-03 21:05:16,094 : INFO : read 40000 reviews
2021-12-03 21:05:18,115 : INFO : read 50000 reviews
2021-12-03 21:05:20,005 : INFO : read 60000 reviews
2021-12-03 21:05:21,820 : INFO : read 70000 reviews
2021-12-03 21:05:23,282 : INFO : read 80000 reviews
2021-12-03 21:05:24,826 : INFO : read 90000 reviews
2021-12-03 21:05:26,394 : INFO : read 100000 reviews
2021-12-03 21:05:27,997 : INFO : read 110000 reviews
2021-12-03 21:05:29,506 : INFO : read 120000 reviews
2021-12-03 21:05:31,093 : INFO : read 130000 reviews
2021-12-03 21:05:32,732 : INFO : read 140000 reviews
2021-12-03 21:05:34,255 : INFO : read 150000 reviews
2021-12-03 21:05:36,308 : INFO : read 160000 reviews
2021-12-03 21:05:37,912

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [4]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2021-12-03 21:06:24,891 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2021-12-03 21:06:24,893 : INFO : collecting all words and their counts
2021-12-03 21:06:24,893 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-12-03 21:06:25,140 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2021-12-03 21:06:25,354 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2021-12-03 21:06:25,600 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2021-12-03 21:06:25,827 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2021-12-03 21:06:26,077 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2021-12-03 21:06:26,329 : INFO : PROGRESS: at sentence #60000, processed 11013726 words, keeping 76786 word types
2021-12-03 21:06:26,536 : INFO : PROGRESS: a

(303499147, 415193580)

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [16]:
w1 = "misinformation"
mis_model = model.wv.most_similar (positive=w1)

In [17]:
mis_model

[('information', 0.6022960543632507),
 ('info', 0.5354797840118408),
 ('responses', 0.5073959231376648),
 ('feedback', 0.500105619430542),
 ('uninformed', 0.47496792674064636),
 ('informations', 0.47363176941871643),
 ('advice', 0.45637232065200806),
 ('interrogated', 0.4491495192050934),
 ('inquiring', 0.44862523674964905),
 ('guidance', 0.4433063864707947)]

In [26]:
closest_word = max(mis_model,key=lambda item:item[1])
print(f"closest word to {w1} is {closest_word}")

closest word to misinformation is ('information', 0.6022960543632507)


In [28]:
def closest_word(a_word_tuple):
    closest = max(a_word_tuple,key=lambda item:item[1])
    return closest

print(f"closest word to {w1} is {closest_word(mis_model)}")

closest word to misinformation is ('information', 0.6022960543632507)


In [5]:

w1 = "dirty"
model.wv.most_similar (positive=w1)


2021-12-03 21:11:58,102 : INFO : precomputing L2-norms of word weight vectors


[('filthy', 0.8662458658218384),
 ('stained', 0.7852855920791626),
 ('unclean', 0.7832462787628174),
 ('smelly', 0.7697746157646179),
 ('grubby', 0.7684239149093628),
 ('dusty', 0.7665725350379944),
 ('dingy', 0.7294144034385681),
 ('soiled', 0.7283311486244202),
 ('grimy', 0.7230293154716492),
 ('disgusting', 0.7189224362373352)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [6]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)


[('courteous', 0.9177724719047546),
 ('friendly', 0.8302420377731323),
 ('cordial', 0.8070728182792664),
 ('professional', 0.7951556444168091),
 ('attentive', 0.7766161561012268),
 ('curteous', 0.7692057490348816)]

In [7]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


[('germany', 0.6914427280426025),
 ('canada', 0.6612461805343628),
 ('spain', 0.6396411061286926),
 ('gaulle', 0.5963788032531738),
 ('mexico', 0.5963702201843262),
 ('england', 0.594928503036499)]

In [8]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


[('horrified', 0.8093999028205872),
 ('astonished', 0.7883663177490234),
 ('amazed', 0.7843524813652039),
 ('dismayed', 0.7592852711677551),
 ('appalled', 0.754350483417511),
 ('stunned', 0.7325351238250732)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [9]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


[('duvet', 0.715851366519928),
 ('quilt', 0.6929425597190857),
 ('mattress', 0.6886236071586609),
 ('blanket', 0.6840144395828247),
 ('matress', 0.6746933460235596),
 ('pillowcase', 0.6726045608520508),
 ('sheets', 0.6438941359519958),
 ('foam', 0.6402884721755981),
 ('pillows', 0.6392641663551331),
 ('pillowcases', 0.6226792335510254)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [10]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

0.7697745

In [11]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

1.0

In [12]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.27420774

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [13]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

/Users/arsanjan/opt/anaconda3/lib/python3.8/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'france'

In [14]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
